In [14]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
sc = SparkSession.builder.getOrCreate()

df = sc.sql('''select 'spark' as hello ''')
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql.types import *
from datetime import datetime

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pyspark.sql.functions as sf
from IPython.display import display


In [15]:
def f(a):
    display(a)
    return a

w = interactive(f, a=widgets.IntSlider(min=1900,max=2015,step=1,value=1996))

print("kies uw jaar met de slider")
display(w)

kies uw jaar met de slider


interactive(children=(IntSlider(value=1996, description='a', max=2015, min=1900), Output()), _dom_classes=('wi…

In [17]:
df = sqlContext.read.load("../data/CO2Uitstoot/annual-co-emissions-per-country.csv", format='com.databricks.spark.csv',header='true',inferscheme='true')
df = df.withColumn("co", df["co"].cast(IntegerType()))
df.createOrReplaceTempView("table1")

table1 = sqlContext.sql("SELECT * FROM table1 WHERE Year == {0} AND Entity != 'World' AND Entity != 'Non KP Annex B' AND Entity != 'Non-OECD' AND Entity != 'OECD' AND Entity != 'KP Annex B' AND Entity != 'Europe' AND Entity != 'Asia'".format(w.result))

In [19]:
import folium
from folium import FeatureGroup, LayerControl, Map, CircleMarker, plugins
comap = folium.Map(location=[51.148077, 04.3389639], zoom_start=4)
plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True).add_to(comap)

import pandas as pd

comap.choropleth(geo_data='../data/world-110m.geojson',
             line_weight=2,
             fill_opacity=0.7,
             data=table1.toPandas(), 
             columns=['Entity', 'co'],
             key_on='properties.name',
             fill_color='YlOrRd',
             legend_name="CO",
             highlight=True
            )

In [20]:
df2 = sqlContext.read.load("../data/temperatuur/GlobalLandTemperaturesByCountry.csv", format='com.databricks.spark.csv',header='true',inferscheme='true')
df2.createOrReplaceTempView("table2")
table2 = sqlContext.sql("SELECT * FROM table2 WHERE AverageTemperature != 'null' AND substring_index(dt,'-',1) == {0}".format(w.result))
table3 = sqlContext.sql("SELECT Country, AVG(AverageTemperature) as AVGTemp FROM table2 GROUP BY Country ORDER BY Country")
table3.createOrReplaceTempView("table3")

In [22]:
import folium
from folium import FeatureGroup, LayerControl, Map, CircleMarker, plugins
tempmap = folium.Map(location=[51.148077, 04.3389639], zoom_start=4)

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True).add_to(tempmap)

import pandas as pd

tempmap.choropleth(geo_data='../data/world-110m.geojson',
             line_weight=2,
             fill_opacity=0.7,
             data=table3.toPandas(), 
             columns=['Country', 'AVGTemp'],
             key_on='properties.name',
             fill_color='YlOrRd',
             legend_name="CO",
             highlight=True
            )

tempmap

In [23]:
comap